<span style="color:#FF7A00; font-size:30px; font-weight:bold;">
BDP Term Project (1조)
</span>

<span style="font-size:20px; font-weight:bold;">
엔카의 국산 중고차 매물별 보험이력 불러오기
</span>

<span style="font-size:17px; font-weight:bold;">
Load Driver
</span>

In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import json

In [6]:
driver = webdriver.Chrome(service= Service(ChromeDriverManager().install()))

<span style="font-size:17px; font-weight:bold;">
페이지 로딩 대기 시간 설정
</span>

In [7]:
wait_time = 0.5

<span style="font-size:17px; font-weight:bold;">
Selenium 호환성 및 버전 확인
</span>

In [8]:
import selenium
print(selenium.__version__)

4.27.1


<span style="font-size:20px; font-weight:bold;">
국산차 브랜드 6사의 중고차 매물 ID별 보험이력 추출<br>
<span style="font-size:15px; font-weight:bold;">
추출대상 : 현대, 제네시스, 기아, 쉐보레, 르노, 쌍용<br>
추출항목 : 매물ID, 차량번호, 번호변경이력, 소유자변경이력, 전손이력, 침수이력, 도난이력, 내차피해이력, 상대차가해이력
</span>

<span style="font-size:20px; font-weight:bold;">
현대자동차
    <span style="font-size:15px; font-weight:bold;">
        "insurance_hyundai.csv" 추출
</span>

In [8]:
import csv
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

# 페이지 순회를 위한 범위 지정 (예: 1~15페이지)
page_start = 1
page_end = 3  # 원하는 페이지 수까지 변경 가능
wait_time = 1  # 페이지 로딩 대기 시간

# 전체 추출 번호를 저장할 리스트
all_numbers = set()

# 크롬 드라이버 초기화
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# 1단계: 차량 번호 추출
for page in range(page_start, page_end + 1):
    try:
        # 페이지 URL 업데이트
        url = f"https://car.encar.com/list/car?page=1&search=%7B%22type%22%3A%22car%22%2C%22action%22%3A%22(And.Hidden.N._.(C.CarType.Y._.Manufacturer.%ED%98%84%EB%8C%80.))%22%2C%22title%22%3A%22%ED%98%84%EB%8C%80%22%2C%22toggle%22%3A%7B%7D%2C%22layer%22%3A%22%22%2C%22sort%22%3A%22MobileModifiedDate%22%7D"
        driver.get(url)
        
        # 페이지 로딩 대기
        time.sleep(wait_time)  # 필요에 따라 WebDriverWait으로 변경 가능
        
        # data-impression 값 추출
        index = 1
        while True:
            try:
                xpath = f"/html/body/div/div[1]/div[4]/div[1]/div[{index}]"
                element = driver.find_element(By.XPATH, xpath)
                data_impression = element.get_attribute("data-impression")
                if data_impression:
                    # 맨 앞 번호만 추출
                    first_number = data_impression.split("|")[0]
                    all_numbers.add(first_number)  # 번호 누적 저장
                index += 1
            except:
                # 현재 페이지의 매물 끝
                break
        
        print(f"페이지 {page} 크롤링 완료, 현재까지 추출된 번호 수: {len(all_numbers)}")
    except Exception as e:
        print(f"페이지 {page}에서 오류 발생: {e}")

print("모든 페이지에서 추출된 번호들:")
print(all_numbers)

# 2단계: 추출된 번호를 이용하여 보험 이력 저장
def fetch_insurance_history_to_csv(driver, vehicle_ids, output_file="insurance_hyundai.csv"):
    """Fetch insurance history for given vehicle IDs and save to CSV."""
    fieldnames = [
        "Vehicle ID", "Car Number", "Number Change History", "Owner Change History",
        "Total Loss History", "Flood Damage History", "Theft History",
        "Insurance Claim (My Car)", "Insurance Claim (Other Car)"
    ]
    
    data = []

    for vehicle_id in vehicle_ids:
        url = f"https://fem.encar.com/cars/report/accident/{vehicle_id}"
        driver.get(url)
        
        # Default values for missing data
        insurance_data = {
            "Vehicle ID": vehicle_id,
            "Car Number": "N/A",
            "Number Change History": "N/A",
            "Owner Change History": "N/A",
            "Total Loss History": "N/A",
            "Flood Damage History": "N/A",
            "Theft History": "N/A",
            "Insurance Claim (My Car)": "N/A",
            "Insurance Claim (Other Car)": "N/A"
        }

        try:
            # Extracting Car Number
            insurance_data["Car Number"] = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, "//*[@id='wrap']/div/div[2]/div[2]/dl/dd[1]"))
            ).text or "N/A"

            # Extracting Number Change History
            insurance_data["Number Change History"] = driver.find_element(
                By.XPATH, "//*[@id='wrap']/div/div[2]/div[2]/div[1]/ul/li[3]/span/span[1]"
            ).text or "N/A"

            # Extracting Owner Change History
            insurance_data["Owner Change History"] = driver.find_element(
                By.XPATH, "//*[@id='wrap']/div/div[2]/div[2]/div[1]/ul/li[3]/span/span[2]"
            ).text or "N/A"

            # Extracting Total Loss History
            insurance_data["Total Loss History"] = driver.find_element(
                By.XPATH, "//*[@id='wrap']/div/div[2]/div[2]/div[1]/ul/li[4]/span/span[1]"
            ).text or "N/A"

            # Extracting Flood Damage History
            insurance_data["Flood Damage History"] = driver.find_element(
                By.XPATH, "//*[@id='wrap']/div/div[2]/div[2]/div[1]/ul/li[4]/span/span[2]"
            ).text or "N/A"

            # Extracting Theft History
            insurance_data["Theft History"] = driver.find_element(
                By.XPATH, "//*[@id='wrap']/div/div[2]/div[2]/div[1]/ul/li[4]/span/span[3]"
            ).text or "N/A"

            # Extracting Insurance Claim (My Car)
            insurance_data["Insurance Claim (My Car)"] = driver.find_element(
                By.XPATH, "//*[@id='wrap']/div/div[2]/div[2]/div[1]/ul/li[5]/span"
            ).text or "N/A"

            # Extracting Insurance Claim (Other Car)
            insurance_data["Insurance Claim (Other Car)"] = driver.find_element(
                By.XPATH, "//*[@id='wrap']/div/div[2]/div[2]/div[1]/ul/li[6]/span"
            ).text or "N/A"

        except Exception as e:
            print(f"Error fetching data for Vehicle ID {vehicle_id}: {e}")

        data.append(insurance_data)

    # Write data to CSV
    with open(output_file, "w", newline="", encoding="utf-8") as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(data)
    
    print(f"csv파일이 {output_file}에 저장되었습니다.")


# Execute the pipeline
try:
    fetch_insurance_history_to_csv(driver, list(all_numbers))
finally:
    driver.quit()

페이지 1 크롤링 완료, 현재까지 추출된 번호 수: 10
페이지 2 크롤링 완료, 현재까지 추출된 번호 수: 20
페이지 3 크롤링 완료, 현재까지 추출된 번호 수: 30
모든 페이지에서 추출된 번호들:
{'38578099', '38548628', '38452993', '38101788', '38070823', '35572276', '38176743', '38443440', '38498950', '37788455', '38503156', '38122585', '38539029', '37264348', '38212599', '38450662', '38347447', '38425494', '38609761', '38269638', '37874807', '38375309', '38389553', '38620905', '38524258', '38352469', '37873111', '38332168', '38518099', '38190253'}
Insurance history saved to insurance_hyundai.csv


<span style="font-size:20px; font-weight:bold;">
제네시스
    <span style="font-size:15px; font-weight:bold;">
        "insurance_genesis.csv" 추출
</span>

In [12]:
# 페이지 순회를 위한 범위 지정 (예: 1~15페이지)
page_start = 1
page_end = 3  # 원하는 페이지 수까지 변경 가능
wait_time = 1  # 페이지 로딩 대기 시간

# 전체 추출 번호를 저장할 리스트
all_numbers = set()

# 크롬 드라이버 초기화
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# 1단계: 차량 번호 추출
for page in range(page_start, page_end + 1):
    try:
        # 페이지 URL 업데이트
        url = f"https://car.encar.com/list/car?page=1&search=%7B%22type%22%3A%22car%22%2C%22action%22%3A%22(And.Hidden.N._.(C.CarType.Y._.Manufacturer.%EC%A0%9C%EB%84%A4%EC%8B%9C%EC%8A%A4.))%22%2C%22title%22%3A%22%EC%A0%9C%EB%84%A4%EC%8B%9C%EC%8A%A4%22%2C%22toggle%22%3A%7B%7D%2C%22layer%22%3A%22%22%2C%22sort%22%3A%22MobileModifiedDate%22%7D"
        driver.get(url)
        
        # 페이지 로딩 대기
        time.sleep(wait_time)  # 필요에 따라 WebDriverWait으로 변경 가능
        
        # data-impression 값 추출
        index = 1
        while True:
            try:
                xpath = f"/html/body/div/div[1]/div[4]/div[1]/div[{index}]"
                element = driver.find_element(By.XPATH, xpath)
                data_impression = element.get_attribute("data-impression")
                if data_impression:
                    # 맨 앞 번호만 추출
                    first_number = data_impression.split("|")[0]
                    all_numbers.add(first_number)  # 번호 누적 저장
                index += 1
            except:
                # 현재 페이지의 매물 끝
                break
        
        print(f"페이지 {page} 크롤링 완료, 현재까지 추출된 번호 수: {len(all_numbers)}")
    except Exception as e:
        print(f"페이지 {page}에서 오류 발생: {e}")

print("모든 페이지에서 추출된 번호들:")
print(all_numbers)

# 2단계: 추출된 번호를 이용하여 보험 이력 저장
def fetch_insurance_history_to_csv(driver, vehicle_ids, output_file="insurance_genesis.csv"):
    """Fetch insurance history for given vehicle IDs and save to CSV."""
    fieldnames = [
        "Vehicle ID", "Car Number", "Number Change History", "Owner Change History",
        "Total Loss History", "Flood Damage History", "Theft History",
        "Insurance Claim (My Car)", "Insurance Claim (Other Car)"
    ]
    
    data = []

    for vehicle_id in vehicle_ids:
        url = f"https://fem.encar.com/cars/report/accident/{vehicle_id}"
        driver.get(url)
        
        # Default values for missing data
        insurance_data = {
            "Vehicle ID": vehicle_id,
            "Car Number": "N/A",
            "Number Change History": "N/A",
            "Owner Change History": "N/A",
            "Total Loss History": "N/A",
            "Flood Damage History": "N/A",
            "Theft History": "N/A",
            "Insurance Claim (My Car)": "N/A",
            "Insurance Claim (Other Car)": "N/A"
        }

        try:
            # Extracting Car Number
            insurance_data["Car Number"] = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, "//*[@id='wrap']/div/div[2]/div[2]/dl/dd[1]"))
            ).text or "N/A"

            # Extracting Number Change History
            insurance_data["Number Change History"] = driver.find_element(
                By.XPATH, "//*[@id='wrap']/div/div[2]/div[2]/div[1]/ul/li[3]/span/span[1]"
            ).text or "N/A"

            # Extracting Owner Change History
            insurance_data["Owner Change History"] = driver.find_element(
                By.XPATH, "//*[@id='wrap']/div/div[2]/div[2]/div[1]/ul/li[3]/span/span[2]"
            ).text or "N/A"

            # Extracting Total Loss History
            insurance_data["Total Loss History"] = driver.find_element(
                By.XPATH, "//*[@id='wrap']/div/div[2]/div[2]/div[1]/ul/li[4]/span/span[1]"
            ).text or "N/A"

            # Extracting Flood Damage History
            insurance_data["Flood Damage History"] = driver.find_element(
                By.XPATH, "//*[@id='wrap']/div/div[2]/div[2]/div[1]/ul/li[4]/span/span[2]"
            ).text or "N/A"

            # Extracting Theft History
            insurance_data["Theft History"] = driver.find_element(
                By.XPATH, "//*[@id='wrap']/div/div[2]/div[2]/div[1]/ul/li[4]/span/span[3]"
            ).text or "N/A"

            # Extracting Insurance Claim (My Car)
            insurance_data["Insurance Claim (My Car)"] = driver.find_element(
                By.XPATH, "//*[@id='wrap']/div/div[2]/div[2]/div[1]/ul/li[5]/span"
            ).text or "N/A"

            # Extracting Insurance Claim (Other Car)
            insurance_data["Insurance Claim (Other Car)"] = driver.find_element(
                By.XPATH, "//*[@id='wrap']/div/div[2]/div[2]/div[1]/ul/li[6]/span"
            ).text or "N/A"

        except Exception as e:
            print(f"Error fetching data for Vehicle ID {vehicle_id}: {e}")

        data.append(insurance_data)

    # Write data to CSV
    with open(output_file, "w", newline="", encoding="utf-8") as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(data)
    
    print(f"csv파일이 {output_file}에 저장되었습니다.")


# Execute the pipeline
try:
    fetch_insurance_history_to_csv(driver, list(all_numbers))
finally:
    driver.quit()

페이지 1 크롤링 완료, 현재까지 추출된 번호 수: 10
페이지 2 크롤링 완료, 현재까지 추출된 번호 수: 19
페이지 3 크롤링 완료, 현재까지 추출된 번호 수: 29
모든 페이지에서 추출된 번호들:
{'38222936', '38314128', '38282894', '38377854', '37813598', '38252155', '38503716', '37987378', '38421022', '38278231', '38529508', '38490395', '38607266', '38420883', '38587464', '38589155', '38579752', '36978745', '38607872', '38579024', '38496320', '38260510', '38320815', '37451703', '38311075', '37542660', '38366721', '38503826', '38378437'}
Error fetching data for Vehicle ID 38278231: Message: 
Stacktrace:
	GetHandleVerifier [0x011033E3+25059]
	(No symbol) [0x0108CDE4]
	(No symbol) [0x00F6BEC3]
	(No symbol) [0x00FAFD86]
	(No symbol) [0x00FAFFCB]
	(No symbol) [0x00FED952]
	(No symbol) [0x00FD1F44]
	(No symbol) [0x00FEB51E]
	(No symbol) [0x00FD1C96]
	(No symbol) [0x00FA3FAC]
	(No symbol) [0x00FA4F3D]
	GetHandleVerifier [0x013F5543+3113795]
	GetHandleVerifier [0x0140A20A+3198986]
	GetHandleVerifier [0x014029E2+3168226]
	GetHandleVerifier [0x011A3250+680016]
	(No symbol) 

<span style="font-size:20px; font-weight:bold;">
기아자동차
    <span style="font-size:15px; font-weight:bold;">
        "insurance_kia.csv" 추출
</span>

In [10]:
# 페이지 순회를 위한 범위 지정 (예: 1~15페이지)
page_start = 1
page_end = 3  # 원하는 페이지 수까지 변경 가능
wait_time = 1  # 페이지 로딩 대기 시간

# 전체 추출 번호를 저장할 리스트
all_numbers = set()

# 크롬 드라이버 초기화
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# 1단계: 차량 번호 추출
for page in range(page_start, page_end + 1):
    try:
        # 페이지 URL 업데이트
        url = f"https://car.encar.com/list/car?page=1&search=%7B%22type%22%3A%22car%22%2C%22action%22%3A%22(And.Hidden.N._.(C.CarType.Y._.Manufacturer.%EA%B8%B0%EC%95%84.))%22%2C%22title%22%3A%22%EA%B8%B0%EC%95%84%22%2C%22toggle%22%3A%7B%7D%2C%22layer%22%3A%22%22%2C%22sort%22%3A%22MobileModifiedDate%22%7D"
        driver.get(url)
        
        # 페이지 로딩 대기
        time.sleep(wait_time)  # 필요에 따라 WebDriverWait으로 변경 가능
        
        # data-impression 값 추출
        index = 1
        while True:
            try:
                xpath = f"/html/body/div/div[1]/div[4]/div[1]/div[{index}]"
                element = driver.find_element(By.XPATH, xpath)
                data_impression = element.get_attribute("data-impression")
                if data_impression:
                    # 맨 앞 번호만 추출
                    first_number = data_impression.split("|")[0]
                    all_numbers.add(first_number)  # 번호 누적 저장
                index += 1
            except:
                # 현재 페이지의 매물 끝
                break
        
        print(f"페이지 {page} 크롤링 완료, 현재까지 추출된 번호 수: {len(all_numbers)}")
    except Exception as e:
        print(f"페이지 {page}에서 오류 발생: {e}")

print("모든 페이지에서 추출된 번호들:")
print(all_numbers)

# 2단계: 추출된 번호를 이용하여 보험 이력 저장
def fetch_insurance_history_to_csv(driver, vehicle_ids, output_file="insurance_kia.csv"):
    """Fetch insurance history for given vehicle IDs and save to CSV."""
    fieldnames = [
        "Vehicle ID", "Car Number", "Number Change History", "Owner Change History",
        "Total Loss History", "Flood Damage History", "Theft History",
        "Insurance Claim (My Car)", "Insurance Claim (Other Car)"
    ]
    
    data = []

    for vehicle_id in vehicle_ids:
        url = f"https://fem.encar.com/cars/report/accident/{vehicle_id}"
        driver.get(url)
        
        # Default values for missing data
        insurance_data = {
            "Vehicle ID": vehicle_id,
            "Car Number": "N/A",
            "Number Change History": "N/A",
            "Owner Change History": "N/A",
            "Total Loss History": "N/A",
            "Flood Damage History": "N/A",
            "Theft History": "N/A",
            "Insurance Claim (My Car)": "N/A",
            "Insurance Claim (Other Car)": "N/A"
        }

        try:
            # Extracting Car Number
            insurance_data["Car Number"] = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, "//*[@id='wrap']/div/div[2]/div[2]/dl/dd[1]"))
            ).text or "N/A"

            # Extracting Number Change History
            insurance_data["Number Change History"] = driver.find_element(
                By.XPATH, "//*[@id='wrap']/div/div[2]/div[2]/div[1]/ul/li[3]/span/span[1]"
            ).text or "N/A"

            # Extracting Owner Change History
            insurance_data["Owner Change History"] = driver.find_element(
                By.XPATH, "//*[@id='wrap']/div/div[2]/div[2]/div[1]/ul/li[3]/span/span[2]"
            ).text or "N/A"

            # Extracting Total Loss History
            insurance_data["Total Loss History"] = driver.find_element(
                By.XPATH, "//*[@id='wrap']/div/div[2]/div[2]/div[1]/ul/li[4]/span/span[1]"
            ).text or "N/A"

            # Extracting Flood Damage History
            insurance_data["Flood Damage History"] = driver.find_element(
                By.XPATH, "//*[@id='wrap']/div/div[2]/div[2]/div[1]/ul/li[4]/span/span[2]"
            ).text or "N/A"

            # Extracting Theft History
            insurance_data["Theft History"] = driver.find_element(
                By.XPATH, "//*[@id='wrap']/div/div[2]/div[2]/div[1]/ul/li[4]/span/span[3]"
            ).text or "N/A"

            # Extracting Insurance Claim (My Car)
            insurance_data["Insurance Claim (My Car)"] = driver.find_element(
                By.XPATH, "//*[@id='wrap']/div/div[2]/div[2]/div[1]/ul/li[5]/span"
            ).text or "N/A"

            # Extracting Insurance Claim (Other Car)
            insurance_data["Insurance Claim (Other Car)"] = driver.find_element(
                By.XPATH, "//*[@id='wrap']/div/div[2]/div[2]/div[1]/ul/li[6]/span"
            ).text or "N/A"

        except Exception as e:
            print(f"Error fetching data for Vehicle ID {vehicle_id}: {e}")

        data.append(insurance_data)

    # Write data to CSV
    with open(output_file, "w", newline="", encoding="utf-8") as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(data)
    
    print(f"csv파일이 {output_file}에 저장되었습니다.")


# Execute the pipeline
try:
    fetch_insurance_history_to_csv(driver, list(all_numbers))
finally:
    driver.quit()

페이지 1 크롤링 완료, 현재까지 추출된 번호 수: 10
페이지 2 크롤링 완료, 현재까지 추출된 번호 수: 20
페이지 3 크롤링 완료, 현재까지 추출된 번호 수: 30
모든 페이지에서 추출된 번호들:
{'38307423', '38434224', '38488177', '37777963', '38539955', '36768728', '38515972', '38472696', '38548498', '38406475', '38334763', '38474269', '38346468', '38009721', '38354300', '38606376', '37962443', '38454566', '38474232', '37068401', '38097033', '38625427', '38214838', '38500955', '38096301', '38395671', '38095546', '38611938', '38321363', '38342238'}
Insurance history saved to insurance_kia.csv


<span style="font-size:20px; font-weight:bold;">
쉐보레(GM대우)
    <span style="font-size:15px; font-weight:bold;">
        "insurance_chevrolet.csv" 추출
</span>

In [11]:
# 페이지 순회를 위한 범위 지정 (예: 1~15페이지)
page_start = 1
page_end = 3  # 원하는 페이지 수까지 변경 가능
wait_time = 1  # 페이지 로딩 대기 시간

# 전체 추출 번호를 저장할 리스트
all_numbers = set()

# 크롬 드라이버 초기화
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# 1단계: 차량 번호 추출
for page in range(page_start, page_end + 1):
    try:
        # 페이지 URL 업데이트
        url = f"https://car.encar.com/list/car?page=1&search=%7B%22type%22%3A%22car%22%2C%22action%22%3A%22(And.Hidden.N._.(C.CarType.Y._.Manufacturer.%EC%89%90%EB%B3%B4%EB%A0%88(GM%EB%8C%80%EC%9A%B0_).))%22%2C%22title%22%3A%22%EC%89%90%EB%B3%B4%EB%A0%88(GM%EB%8C%80%EC%9A%B0)%22%2C%22toggle%22%3A%7B%7D%2C%22layer%22%3A%22%22%2C%22sort%22%3A%22MobileModifiedDate%22%7D"
        driver.get(url)
        
        # 페이지 로딩 대기
        time.sleep(wait_time)  # 필요에 따라 WebDriverWait으로 변경 가능
        
        # data-impression 값 추출
        index = 1
        while True:
            try:
                xpath = f"/html/body/div/div[1]/div[4]/div[1]/div[{index}]"
                element = driver.find_element(By.XPATH, xpath)
                data_impression = element.get_attribute("data-impression")
                if data_impression:
                    # 맨 앞 번호만 추출
                    first_number = data_impression.split("|")[0]
                    all_numbers.add(first_number)  # 번호 누적 저장
                index += 1
            except:
                # 현재 페이지의 매물 끝
                break
        
        print(f"페이지 {page} 크롤링 완료, 현재까지 추출된 번호 수: {len(all_numbers)}")
    except Exception as e:
        print(f"페이지 {page}에서 오류 발생: {e}")

print("모든 페이지에서 추출된 번호들:")
print(all_numbers)

# 2단계: 추출된 번호를 이용하여 보험 이력 저장
def fetch_insurance_history_to_csv(driver, vehicle_ids, output_file="insurance_chevrolet.csv"):
    """Fetch insurance history for given vehicle IDs and save to CSV."""
    fieldnames = [
        "Vehicle ID", "Car Number", "Number Change History", "Owner Change History",
        "Total Loss History", "Flood Damage History", "Theft History",
        "Insurance Claim (My Car)", "Insurance Claim (Other Car)"
    ]
    
    data = []

    for vehicle_id in vehicle_ids:
        url = f"https://fem.encar.com/cars/report/accident/{vehicle_id}"
        driver.get(url)
        
        # Default values for missing data
        insurance_data = {
            "Vehicle ID": vehicle_id,
            "Car Number": "N/A",
            "Number Change History": "N/A",
            "Owner Change History": "N/A",
            "Total Loss History": "N/A",
            "Flood Damage History": "N/A",
            "Theft History": "N/A",
            "Insurance Claim (My Car)": "N/A",
            "Insurance Claim (Other Car)": "N/A"
        }

        try:
            # Extracting Car Number
            insurance_data["Car Number"] = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, "//*[@id='wrap']/div/div[2]/div[2]/dl/dd[1]"))
            ).text or "N/A"

            # Extracting Number Change History
            insurance_data["Number Change History"] = driver.find_element(
                By.XPATH, "//*[@id='wrap']/div/div[2]/div[2]/div[1]/ul/li[3]/span/span[1]"
            ).text or "N/A"

            # Extracting Owner Change History
            insurance_data["Owner Change History"] = driver.find_element(
                By.XPATH, "//*[@id='wrap']/div/div[2]/div[2]/div[1]/ul/li[3]/span/span[2]"
            ).text or "N/A"

            # Extracting Total Loss History
            insurance_data["Total Loss History"] = driver.find_element(
                By.XPATH, "//*[@id='wrap']/div/div[2]/div[2]/div[1]/ul/li[4]/span/span[1]"
            ).text or "N/A"

            # Extracting Flood Damage History
            insurance_data["Flood Damage History"] = driver.find_element(
                By.XPATH, "//*[@id='wrap']/div/div[2]/div[2]/div[1]/ul/li[4]/span/span[2]"
            ).text or "N/A"

            # Extracting Theft History
            insurance_data["Theft History"] = driver.find_element(
                By.XPATH, "//*[@id='wrap']/div/div[2]/div[2]/div[1]/ul/li[4]/span/span[3]"
            ).text or "N/A"

            # Extracting Insurance Claim (My Car)
            insurance_data["Insurance Claim (My Car)"] = driver.find_element(
                By.XPATH, "//*[@id='wrap']/div/div[2]/div[2]/div[1]/ul/li[5]/span"
            ).text or "N/A"

            # Extracting Insurance Claim (Other Car)
            insurance_data["Insurance Claim (Other Car)"] = driver.find_element(
                By.XPATH, "//*[@id='wrap']/div/div[2]/div[2]/div[1]/ul/li[6]/span"
            ).text or "N/A"

        except Exception as e:
            print(f"Error fetching data for Vehicle ID {vehicle_id}: {e}")

        data.append(insurance_data)

    # Write data to CSV
    with open(output_file, "w", newline="", encoding="utf-8") as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(data)
    
    print(f"csv파일이 {output_file}에 저장되었습니다.")


# Execute the pipeline
try:
    fetch_insurance_history_to_csv(driver, list(all_numbers))
finally:
    driver.quit()

페이지 1 크롤링 완료, 현재까지 추출된 번호 수: 10
페이지 2 크롤링 완료, 현재까지 추출된 번호 수: 20
페이지 3 크롤링 완료, 현재까지 추출된 번호 수: 30
모든 페이지에서 추출된 번호들:
{'37727978', '38417147', '38450195', '37982397', '38290543', '37135371', '38371461', '38447624', '38225670', '38290142', '38341640', '38507989', '38108527', '38314252', '38580921', '38130442', '37926114', '38256616', '38616885', '37968731', '37990609', '37590211', '38484445', '37862974', '38414982', '38582626', '38213817', '37779138', '38543613', '38168247'}
Insurance history saved to insurance_chevrolet.csv


<span style="font-size:20px; font-weight:bold;">
르노코리아(삼성)
    <span style="font-size:15px; font-weight:bold;">
        "insurance_renault.csv" 추출
</span>

In [13]:
# 페이지 순회를 위한 범위 지정 (예: 1~15페이지)
page_start = 1
page_end = 3  # 원하는 페이지 수까지 변경 가능
wait_time = 1  # 페이지 로딩 대기 시간

# 전체 추출 번호를 저장할 리스트
all_numbers = set()

# 크롬 드라이버 초기화
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# 1단계: 차량 번호 추출
for page in range(page_start, page_end + 1):
    try:
        # 페이지 URL 업데이트
        url = f"https://car.encar.com/list/car?page=1&search=%7B%22type%22%3A%22car%22%2C%22action%22%3A%22(And.Hidden.N._.(C.CarType.Y._.Manufacturer.%EB%A5%B4%EB%85%B8%EC%BD%94%EB%A6%AC%EC%95%84(%EC%82%BC%EC%84%B1_).))%22%2C%22title%22%3A%22%EB%A5%B4%EB%85%B8%EC%BD%94%EB%A6%AC%EC%95%84(%EC%82%BC%EC%84%B1)%22%2C%22toggle%22%3A%7B%7D%2C%22layer%22%3A%22%22%2C%22sort%22%3A%22MobileModifiedDate%22%7D"
        driver.get(url)
        
        # 페이지 로딩 대기
        time.sleep(wait_time)  # 필요에 따라 WebDriverWait으로 변경 가능
        
        # data-impression 값 추출
        index = 1
        while True:
            try:
                xpath = f"/html/body/div/div[1]/div[4]/div[1]/div[{index}]"
                element = driver.find_element(By.XPATH, xpath)
                data_impression = element.get_attribute("data-impression")
                if data_impression:
                    # 맨 앞 번호만 추출
                    first_number = data_impression.split("|")[0]
                    all_numbers.add(first_number)  # 번호 누적 저장
                index += 1
            except:
                # 현재 페이지의 매물 끝
                break
        
        print(f"페이지 {page} 크롤링 완료, 현재까지 추출된 번호 수: {len(all_numbers)}")
    except Exception as e:
        print(f"페이지 {page}에서 오류 발생: {e}")

print("모든 페이지에서 추출된 번호들:")
print(all_numbers)

# 2단계: 추출된 번호를 이용하여 보험 이력 저장
def fetch_insurance_history_to_csv(driver, vehicle_ids, output_file="insurance_renault.csv"):
    """Fetch insurance history for given vehicle IDs and save to CSV."""
    fieldnames = [
        "Vehicle ID", "Car Number", "Number Change History", "Owner Change History",
        "Total Loss History", "Flood Damage History", "Theft History",
        "Insurance Claim (My Car)", "Insurance Claim (Other Car)"
    ]
    
    data = []

    for vehicle_id in vehicle_ids:
        url = f"https://fem.encar.com/cars/report/accident/{vehicle_id}"
        driver.get(url)
        
        # Default values for missing data
        insurance_data = {
            "Vehicle ID": vehicle_id,
            "Car Number": "N/A",
            "Number Change History": "N/A",
            "Owner Change History": "N/A",
            "Total Loss History": "N/A",
            "Flood Damage History": "N/A",
            "Theft History": "N/A",
            "Insurance Claim (My Car)": "N/A",
            "Insurance Claim (Other Car)": "N/A"
        }

        try:
            # Extracting Car Number
            insurance_data["Car Number"] = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, "//*[@id='wrap']/div/div[2]/div[2]/dl/dd[1]"))
            ).text or "N/A"

            # Extracting Number Change History
            insurance_data["Number Change History"] = driver.find_element(
                By.XPATH, "//*[@id='wrap']/div/div[2]/div[2]/div[1]/ul/li[3]/span/span[1]"
            ).text or "N/A"

            # Extracting Owner Change History
            insurance_data["Owner Change History"] = driver.find_element(
                By.XPATH, "//*[@id='wrap']/div/div[2]/div[2]/div[1]/ul/li[3]/span/span[2]"
            ).text or "N/A"

            # Extracting Total Loss History
            insurance_data["Total Loss History"] = driver.find_element(
                By.XPATH, "//*[@id='wrap']/div/div[2]/div[2]/div[1]/ul/li[4]/span/span[1]"
            ).text or "N/A"

            # Extracting Flood Damage History
            insurance_data["Flood Damage History"] = driver.find_element(
                By.XPATH, "//*[@id='wrap']/div/div[2]/div[2]/div[1]/ul/li[4]/span/span[2]"
            ).text or "N/A"

            # Extracting Theft History
            insurance_data["Theft History"] = driver.find_element(
                By.XPATH, "//*[@id='wrap']/div/div[2]/div[2]/div[1]/ul/li[4]/span/span[3]"
            ).text or "N/A"

            # Extracting Insurance Claim (My Car)
            insurance_data["Insurance Claim (My Car)"] = driver.find_element(
                By.XPATH, "//*[@id='wrap']/div/div[2]/div[2]/div[1]/ul/li[5]/span"
            ).text or "N/A"

            # Extracting Insurance Claim (Other Car)
            insurance_data["Insurance Claim (Other Car)"] = driver.find_element(
                By.XPATH, "//*[@id='wrap']/div/div[2]/div[2]/div[1]/ul/li[6]/span"
            ).text or "N/A"

        except Exception as e:
            print(f"Error fetching data for Vehicle ID {vehicle_id}: {e}")

        data.append(insurance_data)

    # Write data to CSV
    with open(output_file, "w", newline="", encoding="utf-8") as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(data)
    
    print(f"csv파일이 {output_file}에 저장되었습니다.")


# Execute the pipeline
try:
    fetch_insurance_history_to_csv(driver, list(all_numbers))
finally:
    driver.quit()

페이지 1 크롤링 완료, 현재까지 추출된 번호 수: 10
페이지 2 크롤링 완료, 현재까지 추출된 번호 수: 19
페이지 3 크롤링 완료, 현재까지 추출된 번호 수: 28
모든 페이지에서 추출된 번호들:
{'38415845', '38382988', '38583617', '38309892', '38326382', '38546220', '38405753', '38415785', '38545218', '38197802', '37625588', '38105190', '37464020', '38387819', '38571622', '38250343', '38357463', '38347440', '38022568', '38448207', '38618098', '38290811', '38061454', '38599702', '38062835', '38042556', '38271872', '38532942'}
Insurance history saved to insurance_renault.csv


<span style="font-size:20px; font-weight:bold;">
KG모빌리티(쌍용자동차)
    <span style="font-size:15px; font-weight:bold;">
        "insurance_kgmobility.csv" 추출
</span>

In [14]:
# 페이지 순회를 위한 범위 지정 (예: 1~15페이지)
page_start = 1
page_end = 3  # 원하는 페이지 수까지 변경 가능
wait_time = 1  # 페이지 로딩 대기 시간

# 전체 추출 번호를 저장할 리스트
all_numbers = set()

# 크롬 드라이버 초기화
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# 1단계: 차량 번호 추출
for page in range(page_start, page_end + 1):
    try:
        # 페이지 URL 업데이트
        url = f"https://car.encar.com/list/car?page=1&search=%7B%22type%22%3A%22car%22%2C%22action%22%3A%22(And.Hidden.N._.(C.CarType.Y._.Manufacturer.KG%EB%AA%A8%EB%B9%8C%EB%A6%AC%ED%8B%B0(%EC%8C%8D%EC%9A%A9_).))%22%2C%22title%22%3A%22KG%EB%AA%A8%EB%B9%8C%EB%A6%AC%ED%8B%B0(%EC%8C%8D%EC%9A%A9)%22%2C%22toggle%22%3A%7B%7D%2C%22layer%22%3A%22%22%2C%22sort%22%3A%22MobileModifiedDate%22%7D"
        driver.get(url)
        
        # 페이지 로딩 대기
        time.sleep(wait_time)  # 필요에 따라 WebDriverWait으로 변경 가능
        
        # data-impression 값 추출
        index = 1
        while True:
            try:
                xpath = f"/html/body/div/div[1]/div[4]/div[1]/div[{index}]"
                element = driver.find_element(By.XPATH, xpath)
                data_impression = element.get_attribute("data-impression")
                if data_impression:
                    # 맨 앞 번호만 추출
                    first_number = data_impression.split("|")[0]
                    all_numbers.add(first_number)  # 번호 누적 저장
                index += 1
            except:
                # 현재 페이지의 매물 끝
                break
        
        print(f"페이지 {page} 크롤링 완료, 현재까지 추출된 번호 수: {len(all_numbers)}")
    except Exception as e:
        print(f"페이지 {page}에서 오류 발생: {e}")

print("모든 페이지에서 추출된 번호들:")
print(all_numbers)

# 2단계: 추출된 번호를 이용하여 보험 이력 저장
def fetch_insurance_history_to_csv(driver, vehicle_ids, output_file="insurance_kgmobility.csv"):
    """Fetch insurance history for given vehicle IDs and save to CSV."""
    fieldnames = [
        "Vehicle ID", "Car Number", "Number Change History", "Owner Change History",
        "Total Loss History", "Flood Damage History", "Theft History",
        "Insurance Claim (My Car)", "Insurance Claim (Other Car)"
    ]
    
    data = []

    for vehicle_id in vehicle_ids:
        url = f"https://fem.encar.com/cars/report/accident/{vehicle_id}"
        driver.get(url)
        
        # Default values for missing data
        insurance_data = {
            "Vehicle ID": vehicle_id,
            "Car Number": "N/A",
            "Number Change History": "N/A",
            "Owner Change History": "N/A",
            "Total Loss History": "N/A",
            "Flood Damage History": "N/A",
            "Theft History": "N/A",
            "Insurance Claim (My Car)": "N/A",
            "Insurance Claim (Other Car)": "N/A"
        }

        try:
            # Extracting Car Number
            insurance_data["Car Number"] = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, "//*[@id='wrap']/div/div[2]/div[2]/dl/dd[1]"))
            ).text or "N/A"

            # Extracting Number Change History
            insurance_data["Number Change History"] = driver.find_element(
                By.XPATH, "//*[@id='wrap']/div/div[2]/div[2]/div[1]/ul/li[3]/span/span[1]"
            ).text or "N/A"

            # Extracting Owner Change History
            insurance_data["Owner Change History"] = driver.find_element(
                By.XPATH, "//*[@id='wrap']/div/div[2]/div[2]/div[1]/ul/li[3]/span/span[2]"
            ).text or "N/A"

            # Extracting Total Loss History
            insurance_data["Total Loss History"] = driver.find_element(
                By.XPATH, "//*[@id='wrap']/div/div[2]/div[2]/div[1]/ul/li[4]/span/span[1]"
            ).text or "N/A"

            # Extracting Flood Damage History
            insurance_data["Flood Damage History"] = driver.find_element(
                By.XPATH, "//*[@id='wrap']/div/div[2]/div[2]/div[1]/ul/li[4]/span/span[2]"
            ).text or "N/A"

            # Extracting Theft History
            insurance_data["Theft History"] = driver.find_element(
                By.XPATH, "//*[@id='wrap']/div/div[2]/div[2]/div[1]/ul/li[4]/span/span[3]"
            ).text or "N/A"

            # Extracting Insurance Claim (My Car)
            insurance_data["Insurance Claim (My Car)"] = driver.find_element(
                By.XPATH, "//*[@id='wrap']/div/div[2]/div[2]/div[1]/ul/li[5]/span"
            ).text or "N/A"

            # Extracting Insurance Claim (Other Car)
            insurance_data["Insurance Claim (Other Car)"] = driver.find_element(
                By.XPATH, "//*[@id='wrap']/div/div[2]/div[2]/div[1]/ul/li[6]/span"
            ).text or "N/A"

        except Exception as e:
            print(f"Error fetching data for Vehicle ID {vehicle_id}: {e}")

        data.append(insurance_data)

    # Write data to CSV
    with open(output_file, "w", newline="", encoding="utf-8") as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(data)
    
    print(f"csv파일이 {output_file}에 저장되었습니다.")


# Execute the pipeline
try:
    fetch_insurance_history_to_csv(driver, list(all_numbers))
finally:
    driver.quit()

페이지 1 크롤링 완료, 현재까지 추출된 번호 수: 10
페이지 2 크롤링 완료, 현재까지 추출된 번호 수: 20
페이지 3 크롤링 완료, 현재까지 추출된 번호 수: 30
모든 페이지에서 추출된 번호들:
{'38623495', '37310300', '38623525', '38425489', '38595866', '38429798', '38356850', '37731767', '38568055', '38508535', '38589035', '37433401', '37906779', '38583696', '38554243', '38531460', '38495264', '38429748', '38463065', '38475268', '37916152', '38289842', '38158514', '38356512', '37944712', '37294393', '38296351', '38450749', '37259730', '38306527'}
Insurance history saved to insurance_kgmobility.csv


<span style="font-size:22px; font-weight:bold;">
제조사 6사 csv 병합
    <span style="font-size:15px; font-weight:bold;">
        "insurance_all.csv" 추출
</span>

In [15]:
import pandas as pd

# 병합할 파일 리스트
file_list = [
    "insurance_hyundai.csv",
    "insurance_genesis.csv",
    "insurance_kia.csv",
    "insurance_chevrolet.csv",
    "insurance_renault.csv",
    "insurance_kgmobility.csv"
]

# 파일을 읽어와서 하나의 DataFrame으로 합치기
combined_df = pd.concat([pd.read_csv(file) for file in file_list], ignore_index=True)

# 합친 CSV 파일 저장
combined_df.to_csv("insurance_all.csv", index=False, encoding="utf-8")
print(f"csv파일이 {output_file}에 병합되어 저장되었습니다.")

모든 csv파일이 'insurance_all.csv'에 병합되어 저장되었습니다.
